In [13]:
import numpy as np
from DecisionTree import DecisionTreeID3
from CrossValidation import *
from helper_functions import _gini, _entropy, _misclass
from metrics import err_mis
import pandas as pd


In [13]:
#shell scripts for downloading the data and placing it in a corresponding directory
!mkdir CAR 
!curl -o CAR/data "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
!curl -o CAR/description "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.names"
#download the description and display it here.
!cat CAR/description

Un sous-répertoire ou un fichier CAR existe déjà.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51867  100 51867    0     0  80289      0 --:--:-- --:--:-- --:--:-- 80289
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3097  100  3097    0     0   9356      0 --:--:-- --:--:-- --:--:--  9328
'cat' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [ ]:
# csv-file has no header, so we define it manually
col_names = ['price_buy', 'price_main', 'n_doors', 'n_persons', 'lug_boot', 'safety', 'recommendation']
df = pd.read_csv("./CAR/data", header=None, names=col_names)


In [18]:
df

,price_buy,price_main,n_doors,n_persons,lug_boot,safety,recommendation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [5]:
# All attributes are categorical - a mix of strings and integers.
# We simply map the categorical values of each attribute to a set of distinct integers
ai2an_map = col_names
ai2aiv2aivn_map = []
enc_cols = []
for col in df.columns:
    df[col] = df[col].astype('category')
    a = np.array(df[col].cat.codes.values).reshape((-1,1))
    enc_cols.append(a)
    ai2aiv2aivn_map.append(list(df[col].cat.categories.values))

    
    
# Get the data as numpy 2d-matrix (n_samples, n_features)
dataset = np.hstack(enc_cols)
X, y = dataset[:,:6], dataset[:,6]
print(X.shape, y.shape)

(1728, 6) (1728,)


In [6]:
tree = DecisionTreeID3()
tree.fit(X, y, verbose=0)

In [8]:
y_p = tree.predict(X)
err = err_mis(y, y_p)
print ("Unsurprisingly, the error on the training set is very low: %f" % err)

Unsurprisingly, the error on the training set is very low: 0.000000


In [14]:
impurity_measures = [_gini, _entropy, _misclass]
k = 10

folds = getKFolds(X.shape[0], k)
#folds = getBootstrapFolds(X.shape[0], k, train_fraction=0.9)

for imp in impurity_measures:
    err_tr = 0.
    err_te = 0.
    for i in range(k):
        idx_tr, idx_te = folds[i]
        
        _X_tr = X[idx_tr]
        _y_tr = y[idx_tr]
        _X_te = X[idx_te]
        _y_te = y[idx_te]
        
        DecisionTreeID3(criterion=imp)
        tree.fit(_X_tr, _y_tr, verbose=0)
        
        y_tr_p = tree.predict(_X_tr)
        y_te_p = tree.predict(_X_te)
        err_tr += err_mis(_y_tr, y_tr_p)
        err_te += err_mis(_y_te, y_te_p)
        
    print ("%s: Average training error %f;Average test error %f" % (imp, err_tr/k, err_te/k))
      

<function _gini at 0x000001C17E11C820>: Average training error 0.000000;Average test error 0.060465
<function _entropy at 0x000001C17E11C8B0>: Average training error 0.000000;Average test error 0.060465
<function _misclass at 0x000001C17E11C940>: Average training error 0.000000;Average test error 0.060465
